In [1]:
# main.py

from config import *
from utils.seed import set_seed
from data.splits import create_splits
from data.transforms import get_transforms
from data.datamodule import ImageDataModule
from models.efficientnet import EfficientNetLit

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torch.utils.data import WeightedRandomSampler

import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


def main():
    set_seed(SEED)

    train_df, val_df, test_df, classes = create_splits(
        metadata_csv= METADATA_DIR,
        images_dir= METADATA_DIR,
        seed=SEED,
        test_size=TEST_SIZE,
        val_size=VAL_SIZE,
    )

    print(
        len(train_df),
        len(val_df),
        len(test_df),
        train_df.lesion_id.nunique(),
        val_df.lesion_id.nunique(),
        test_df.lesion_id.nunique(),
    )

    print("TRAIN class distribution:")
    print(train_df["label"].value_counts().sort_index())

    # считаем количество примеров каждого класса
    counts = train_df["label"].value_counts().sort_index()
    
    # веса классов (обратная частота)
    class_weights_sampler = 1.0 / counts
    
    # вес для каждого сэмпла train датасета
    sample_weights = train_df["label"].map(class_weights_sampler).values
    sample_weights = torch.tensor(sample_weights, dtype=torch.double)
    
    # sampler
    train_sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )

    labels = train_df["label"].values
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(labels),
        y=labels
    )
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    train_tfms, val_tfms, weights = get_transforms()

    datamodule = ImageDataModule(
        train_df,
        val_df,
        test_df,
        train_tfms,
        val_tfms,
        BATCH_SIZE,
        NUM_WORKERS,
        train_sampler=train_sampler,
    )

    model = EfficientNetLit(
        num_classes=len(classes),
        lr_classifier=LR_CLASSIFIER,
        lr_backbone=LR_BACKBONE,
        lr_backbone_finetune=LR_BACKBONE_FINETUNE,
        weight_decay=WEIGHT_DECAY,
        finetune_epoch=FINETUNE_EPOCH,
        min_lr=MIN_LR,
        weights=weights,
        class_weights=class_weights,
    )


    logger = CSVLogger("logs", name="efficientnet")

    early_stop = pl.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=20,
        mode="min",
        verbose=True,
    )

    trainer = pl.Trainer(
        max_epochs=EPOCHS,
        accelerator="auto",
        devices="auto",
        logger=logger,
        callbacks=[
            ModelCheckpoint(
                monitor="val_macro_recall",
                mode="max",
                save_top_k=1,
                filename="epoch{epoch}-recall{val_macro_recall:.4f}",
            ),
            early_stop,
        ],
    )

    trainer.fit(model, datamodule=datamodule)


if __name__ == "__main__":
    main()


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


11507 2657 3475 3218 805 1006
TRAIN class distribution:
label
0    2406
1     103
2     405
3    2639
4    3644
5     206
6     848
7     171
8     708
9     377
Name: count, dtype: int64


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model            │ EfficientNet       │  4.0 M │ train │     0 │
│ 1 │ criterion        │ CrossEntropyLoss   │      0 │ train │     0 │
│ 2 │ val_macro_recall │ MulticlassRecall   │      0 │ train │     0 │
│ 3 │ val_balanced_acc │ MulticlassAccuracy │      0 │ train │     0 │
└───┴──────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 3.7 M                                                                                            
Non-trainable params: 308 K                                                                                        
Total params: 4.0 M                                                                                                
Total estimated model params size (MB): 16                                                                         
Modules in train mode: 340                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

Metric val_loss improved. New best score: 1.972
Metric val_loss improved by 0.062 >= min_delta = 0.0. New best score: 1.910
Metric val_loss improved by 0.019 >= min_delta = 0.0. New best score: 1.890
Monitored metric val_loss did not improve in the last 20 records. Best score: 1.890. Signaling Trainer to stop.
